In [ ]:
import numpy as np

import pandas as pd
from datetime import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker


# myClass 
import import_ipynb
import helper_general as my
import helper_ProcessingResults as myPR

In [ ]:
# Per calcolare media tra run (o subrun) 
def cumMean(dfList, TimeAttribute, att, sampleFreq, legend=None):
    temp = []
    for df in dfList:
        df = df.resample(sampleFreq, on=TimeAttribute)[att].agg({'count'})#.cumsum()# se qui-> Nan influenzano
        temp.append(df)
    return pd.concat(temp, axis=1).mean(axis=1).cumsum() #mean considera Nan -> mean([1,Nan,3])= (1+3)/2
#t = cumMean(df_runs, 'run_totTime', 'ID_Run', '5min', file_id).to_frame()

In [ ]:
def _subPlot(ax, df_toPlot, xAtt, yAtt, linestyle, color=None): 
    # Plot:-----------------------
    for y in yAtt:
        #ax.plot_date(df_toPlot[xAtt], df_toPlot[y], linestyle=linestyle, color=color)#,marker# markersize=2)# plot
        ax.plot_date(df_toPlot[xAtt], df_toPlot[y], linestyle, color=color)

def _subAxsis_TimeBasedLabels(ax, ylabel='', title='', legend=None):
    # Labels:-----------------------
    ax.legend(legend) if legend is not None else None
    ax.set(xlabel='Hours [h]', ylabel=ylabel, title=title)
    ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(0,24,2)))
    ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0,24*60,20)))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax.grid()

# Load + processing File

In [ ]:
sourceDir = "./data_raw/"
filename = ['report_RCE.txt','report_RSE.txt','report_RCA.txt','report_RSA.txt']#,'Report_RC_ud.txt']
report_name = ['RCE','RSE','RCA','RSA'] #NB: stesso ordine dei file caricati
outputDir = "./graphs/"

In [ ]:
df_runs, df_subRuns = [],[]
for f in [sourceDir+f for f in filename]:
    a,b = myPR.loadReport_TEST(f)
    df_runs.append(a.drop(columns=['lenghtConstr','run_startTime']))
    df_subRuns.append(b)

In [ ]:
for df in df_runs:
    df['state'] = df['state'].map(lambda x: 'Confident' if x=='Supported' else x)
display(df_runs[0].head(2))
display(df_subRuns[0].head(2))

In [ ]:
# Per calcolare media tra run (o subrun) 
def cumMean(dfList, TimeAttribute, att, sampleFreq, legend=None):
    temp = []
    for df in dfList:
        df = df.resample(sampleFreq, on=TimeAttribute)[att].agg({'count'})#.cumsum()# se qui-> Nan influenzano
        temp.append(df)
    return pd.concat(temp, axis=1).mean(axis=1).cumsum() #mean considera Nan -> mean([1,Nan,3])= (1+3)/2
#t = cumMean(df_runs, 'run_totTime', 'ID_Run', '5min', file_id).to_frame()

## Grafico 1: N° SubRun over time between methods

### Costruzione dataframe con il solo scopo di construire i grafici più comodamente

In [ ]:
# RCE -> R: Random child, C: Complete digit priority, E: saturate Effective
# RSE -> R: Random child, S: Simple digit priority,   E: saturate Effective
# RCA -> R: Random child, C: Complete digit priority, A: saturate All
# RSA -> R: Random child, S: Simple digit priority  , A: saturate All

In [ ]:
# porto tutti i grafici nella stessa freq temp + ridurre punti del grafo
toPlot=[]
for df, name in zip(df_subRuns, report_name):
    toPlot.append( df.resample('5min', on='subRun_totTime')['subRun_time'].agg({'count'}).cumsum().rename(columns={'count':name}))
toPlot = pd.concat(toPlot, axis=1).reset_index()
my.Display([toPlot.head(3),toPlot.tail(3)], axis=1)

In [ ]:
df_toPlot = toPlot
title_fig = f'Comparison of the number of subrun over time'
legend_fig = report_name

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6,4)) 
_subPlot(axes, toPlot, 'subRun_totTime', report_name, '-')
_subAxsis_TimeBasedLabels(axes, ylabel='N° subrun', title= title_fig, legend=legend_fig)
fig.tight_layout()
fig.savefig(outputDir+title_fig.replace(' ','_').replace(':','')+'.svg', format='svg', dpi=1200)

## Grafico 2: Avg SubRun over time between methods

In [ ]:
toPlot_avgNum=[]
for df, name in zip(df_subRuns, report_name):
    toPlot_avgNum.append( df.resample('10min', on='subRun_totTime')['ID_Run','ID_SubRun']
                      .agg({'ID_Run':set, 'ID_SubRun':'count'})
                      .apply(lambda row: row['ID_SubRun']/len(row['ID_Run']), axis=1).rename(name))
toPlot_avgNum = pd.concat(toPlot_avgNum, axis=1).reset_index()
#my.Display([toPlot_avg.head(3),toPlot_avg.tail(3)], axis=0)

In [ ]:
df_toPlot = toPlot_avgNum
title_fig = f'Comparison of the number of averange subrun over time'
legend_fig = report_name

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,5)) 
_subPlot(axes, df_toPlot.dropna(), 'subRun_totTime', report_name, '-')
_subAxsis_TimeBasedLabels(axes, ylabel='N° avg subrun', title= title_fig, legend=legend_fig)
fig.tight_layout()
fig.savefig(outputDir+title_fig.replace(' ','_').replace(':','')+'.svg', format='svg', dpi=1200)

## Grafico 3: Avg SubRun durate over time between methods

In [ ]:
toPlot_avgTime=[]
zero = datetime.strptime( "00:00", "%H:%M") 
for df, name in zip(df_subRuns, report_name):
    toPlot_avgTime.append( df.resample('10min', on='subRun_totTime')['ID_SubRun','subRun_time']
                      .agg({'ID_SubRun':'count', 'subRun_time':'sum'})
                      .apply(lambda row: row['subRun_time']/row['ID_SubRun']+zero, axis=1).rename(name))
toPlot_avgTime = pd.concat(toPlot_avgTime, axis=1).reset_index()
#my.Display([toPlot_avgTime.head(3),toPlot_avgTime.tail(3)], axis=0)

In [ ]:
df_toPlot = toPlot_avgTime
title_fig = f'Comparison of the averange time of subrun over time'
legend_fig = report_name

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,5)) 
_subPlot(axes, df_toPlot.dropna(), 'subRun_totTime', report_name, '-')
_subAxsis_TimeBasedLabels(axes, ylabel='Avg time for subrun [s]', title= title_fig, legend=legend_fig)
fig.tight_layout()
fig.savefig(outputDir+title_fig.replace(' ','_').replace(':','')+'.svg', format='svg', dpi=1200)

## Grafico 4: Avg duration of SubRun between methods

In [ ]:
title= "Subrun duration's frequence"
legend = report_name


toPlot_avg = []
zero = datetime.strptime( "00:00", "%H:%M") 
for df, name in zip(df_subRuns, report_name):
    df = df.copy() # non sovrascriverer dati originali
    df['subRun_time'] = df['subRun_time']+ zero
    toPlot_avg.append( df.sort_values('subRun_time').resample('1s', on='subRun_time')['ID_SubRun'].agg({'count'}).rename(columns={'count':name}))
toPlot_avg = pd.concat(toPlot_avg, axis=1).reset_index()
#my.Display([toPlot_avg.head(3),toPlot_avg.tail(3)], axis=1)   

#NB: mask nan values per plot !!!!
for rn in report_name:
    toPlot_avg[rn] = toPlot_avg[rn]
    toPlot_avg[rn] = np.ma.masked_where( pd.isna(toPlot_avg[rn]), toPlot_avg[rn])
#my.Display([toPlot_avg.head(3),toPlot_avg.tail(3)], axis=1)

# plot:-------------------------------
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
toPlot_avg = toPlot_avg.head(13).set_index('subRun_time') # NB RIDOTTO GRAFO -> Tralascio coda (val poco influenti)
toPlot_avg.plot.bar(ax=axes, figsize=(16,5))

# labels:---------------
axes.set(xlabel='Duration [s]', ylabel='Frequence of subrun', title=title)
axes.legend(legend)

# axis
ticklabels = [item.strftime('%M:%S') for item in toPlot_avg.index]
axes.set_xticklabels(ticklabels, rotation=0)
axes.grid()

# save
fig.savefig(outputDir+title.replace(' ','_').replace(':','').replace("''",'')+'.svg', format='svg', dpi=1200)